In [9]:
import time
import csv
from urllib.parse import urljoin

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, StaleElementReferenceException, ElementClickInterceptedException

In [ ]:
#get links from all previous matches
import re
import time
import csv
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import JavascriptException


START_URL = "https://www.sofascore.com/tournament/football/singapore/premier-league/634"
SEASON_IDS = [78478, 59708, 48772]


def build_driver(headless=False):
    opts = Options()
    if headless:
        opts.add_argument("--headless=new")
    opts.add_argument("--window-size=1400,900")
    opts.add_argument("--disable-gpu")
    opts.add_argument("--no-sandbox")
    opts.add_argument("--disable-dev-shm-usage")
    opts.add_argument("--lang=en-US")
    return webdriver.Chrome(options=opts)


def parse_event_id(url: str):
    m = re.search(r"#id:(\d+)", url)
    return int(m.group(1)) if m else None


def js_collect_links(driver):
    return driver.execute_script("""
        const out = [];
        const anchors = document.querySelectorAll("a[href*='/football/match/']");
        for (const a of anchors) {
            const href = a.getAttribute("href");
            if (!href) continue;
            if (href.includes("#id:")) out.push(new URL(href, location.origin).href);
        }
        return Array.from(new Set(out));
    """)


def wait_match_list_present(driver, timeout=20):
    wait = WebDriverWait(driver, timeout)
    wait.until(lambda d: d.execute_script(
        "return document.querySelectorAll(\"a[href*='/football/match/']\").length"
    ) > 0)


def js_click_prev_round(driver):
    """
    Click the LEFT arrow button that loads older rounds.
    Returns True if clicked, False if not found or disabled.
    Uses a heuristic: find a button with an SVG path that matches the left-chevron.
    """
    try:
        return driver.execute_script(r"""
            // left-chevron path signature (from your HTML)
            const sig = "M6 11.99 14.058 4l1.432 1.42-6.636 6.57 6.646 6.6L14.078 20z";
            const paths = Array.from(document.querySelectorAll("button svg path"));

            for (const p of paths) {
                const d = (p.getAttribute("d") || "").trim();
                if (d === sig) {
                    const btn = p.closest("button");
                    if (!btn) continue;

                    // check disabled states
                    const ariaDisabled = btn.getAttribute("aria-disabled");
                    const isDisabled = btn.disabled || ariaDisabled === "true" || btn.hasAttribute("disabled");
                    if (isDisabled) return false;

                    btn.scrollIntoView({block: "center"});
                    btn.click();
                    return true;
                }
            }
            return false;
        """)
    except JavascriptException:
        return False


def scrape_one_season_by_clicking_prev(driver, season_id, max_clicks=80, pause=0.9, stop_when_no_new=5):
    driver.get(f"{START_URL}#id:{season_id}")
    wait_match_list_present(driver)

    all_links = set()
    no_new = 0

    # initial collect
    for u in js_collect_links(driver):
        all_links.add(u)

    for i in range(max_clicks):
        before = len(all_links)

        clicked = js_click_prev_round(driver)
        if not clicked:
            print(f"season {season_id} | prev button disabled/not found -> stop")
            break

        time.sleep(pause)

        # after click, collect links again
        for u in js_collect_links(driver):
            all_links.add(u)

        after = len(all_links)
        print(f"season {season_id} | click {i+1} | links {after}")

        if after == before:
            no_new += 1
        else:
            no_new = 0

        if no_new >= stop_when_no_new:
            print(f"season {season_id} | no new links for {stop_when_no_new} clicks -> stop")
            break

    rows = []
    for u in sorted(all_links):
        eid = parse_event_id(u)
        if eid is not None:
            rows.append({"season_id": season_id, "event_id": eid, "match_url": u})

    # de-dupe by (season_id, event_id)
    seen = set()
    uniq = []
    for r in rows:
        key = (r["season_id"], r["event_id"])
        if key not in seen:
            seen.add(key)
            uniq.append(r)

    return uniq


def save_csv(rows, out_path="spl_match_links_3_seasons.csv"):
    with open(out_path, "w", newline="", encoding="utf-8") as f:
        w = csv.DictWriter(f, fieldnames=["season_id", "event_id", "match_url"])
        w.writeheader()
        w.writerows(rows)
    print("saved:", out_path, "rows:", len(rows))


if __name__ == "__main__":
    driver = build_driver(headless=False)
    try:
        all_rows = []
        for sid in SEASON_IDS:
            all_rows.extend(scrape_one_season_by_clicking_prev(driver, sid))

        # global de-dupe
        seen = set()
        final_rows = []
        for r in all_rows:
            key = (r["season_id"], r["event_id"])
            if key not in seen:
                seen.add(key)
                final_rows.append(r)

        print("TOTAL:", len(final_rows))
        print("SAMPLE:", final_rows[:5])
        save_csv(final_rows)

    finally:
        driver.quit()


WebDriverException: Message: unknown error: cannot determine loading status
from target frame detached
  (Session info: chrome=143.0.7499.41)
Stacktrace:
0   chromedriver                        0x0000000104d2ad7c cxxbridge1$str$ptr + 3028012
1   chromedriver                        0x0000000104d22c3c cxxbridge1$str$ptr + 2994924
2   chromedriver                        0x000000010481e980 _RNvCsgXDX2mvAJAg_7___rustc35___rust_no_alloc_shim_is_unstable_v2 + 73784
3   chromedriver                        0x0000000104809394 cxxbridge1$string$len + 3284
4   chromedriver                        0x000000010480721c chromedriver + 209436
5   chromedriver                        0x0000000104807db4 chromedriver + 212404
6   chromedriver                        0x00000001048151e4 _RNvCsgXDX2mvAJAg_7___rustc35___rust_no_alloc_shim_is_unstable_v2 + 34972
7   chromedriver                        0x000000010482b0c8 _RNvCsgXDX2mvAJAg_7___rustc35___rust_no_alloc_shim_is_unstable_v2 + 124800
8   chromedriver    

In [ ]:
#get stats from all events
import os
import re
import json
import time
import random
import csv
import unicodedata
import pandas as pd

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait

IN_PATH = "/Users/chenjingxiang/PowerRanking/spl_match_links_3_seasons.csv"
OUT_PATH = "/Users/chenjingxiang/PowerRanking/spl_stats_3_seasons.csv"
CACHE_DIR = "/Users/chenjingxiang/PowerRanking/.sofascore_stats_cache"

STATS_URL_TMPL = "https://www.sofascore.com/api/v1/event/{event_id}/statistics"

RESUME = False          # set True if you want to continue from existing OUT_PATH
HEADLESS = False
MAX_EVENTS = None       # set e.g. 20 to test quickly


def build_driver(headless=False):
    opts = Options()
    if headless:
        opts.add_argument("--headless=new")
    opts.add_argument("--window-size=1400,900")
    opts.add_argument("--disable-gpu")
    opts.add_argument("--no-sandbox")
    opts.add_argument("--disable-dev-shm-usage")
    opts.add_argument("--lang=en-US")
    return webdriver.Chrome(options=opts)


def selenium_fetch_json(driver, url, timeout=20, debug=False):
    driver.get(url)
    wait = WebDriverWait(driver, timeout)

    try:
        wait.until(lambda d: len(d.find_elements("tag name", "pre")) > 0)
        txt = driver.find_element("tag name", "pre").text
        if debug:
            print("PRE_HEAD:", txt[:120].replace("\n", " "))
        return json.loads(txt)
    except Exception:
        try:
            txt = driver.find_element("tag name", "body").text
            if debug:
                print("BODY_HEAD:", txt[:120].replace("\n", " "))
            return json.loads(txt)
        except Exception:
            if debug:
                print("Could not parse JSON. Current URL:", driver.current_url)
            return None


def slugify_stat_name(name: str) -> str:
    s = unicodedata.normalize("NFKD", name).encode("ascii", "ignore").decode("ascii")
    s = s.lower().strip()
    s = s.replace("&", " and ")
    s = re.sub(r"[^a-z0-9]+", "_", s)
    s = re.sub(r"_+", "_", s).strip("_")
    return s


def flatten_sofascore_statistics(stats_json: dict, period: str = "ALL") -> dict:
    row = {}
    if not stats_json:
        return row

    blocks = stats_json.get("statistics", [])
    block = next((b for b in blocks if b.get("period") == period), blocks[0] if blocks else None)
    if not block:
        return row

    for group in block.get("groups", []):
        for item in group.get("statisticsItems", []):
            base = slugify_stat_name(item.get("name", ""))

            # collision guard
            if f"home_{base}" in row or f"away_{base}" in row:
                k = item.get("key")
                if k:
                    base = f"{base}__{slugify_stat_name(k)}"
                else:
                    c = 2
                    while f"home_{base}_{c}" in row:
                        c += 1
                    base = f"{base}_{c}"

            home_val = item.get("homeValue", item.get("home"))
            away_val = item.get("awayValue", item.get("away"))

            row[f"home_{base}"] = home_val
            row[f"away_{base}"] = away_val

            if "homeTotal" in item:
                row[f"home_{base}_total"] = item.get("homeTotal")
            if "awayTotal" in item:
                row[f"away_{base}_total"] = item.get("awayTotal")

    return row


def ensure_dir(path: str):
    os.makedirs(path, exist_ok=True)


def cache_path(event_id: int) -> str:
    return os.path.join(CACHE_DIR, f"{event_id}.json")


def load_cached(event_id: int):
    p = cache_path(event_id)
    if os.path.exists(p):
        try:
            with open(p, "r", encoding="utf-8") as f:
                return json.load(f)
        except Exception:
            return None
    return None


def save_cached(event_id: int, data: dict):
    p = cache_path(event_id)
    try:
        with open(p, "w", encoding="utf-8") as f:
            json.dump(data, f)
    except Exception:
        pass


def write_rows(rows, out_path):
    cols = set()
    for r in rows:
        cols.update(r.keys())

    base_cols = [c for c in ["season_id", "event_id", "fetch_ok"] if c in cols]
    other_cols = sorted([c for c in cols if c not in base_cols])
    fieldnames = base_cols + other_cols

    with open(out_path, "w", newline="", encoding="utf-8") as f:
        w = csv.DictWriter(f, fieldnames=fieldnames)
        w.writeheader()
        w.writerows(rows)


def main():
    ensure_dir(CACHE_DIR)

    df = pd.read_csv(IN_PATH)

    if "event_id" not in df.columns and "match_url" in df.columns:
        df["event_id"] = df["match_url"].str.extract(r"#id:(\d+)").astype("Int64")

    df = df.dropna(subset=["event_id"]).copy()
    df["event_id"] = df["event_id"].astype(int)

    season_map = None
    if "season_id" in df.columns:
        season_map = df.drop_duplicates("event_id").set_index("event_id")["season_id"].to_dict()

    event_ids = df["event_id"].drop_duplicates().tolist()
    print("loaded event_ids:", len(event_ids))

    if MAX_EVENTS is not None:
        event_ids = event_ids[:MAX_EVENTS]
        print("testing only first:", len(event_ids))

    done_ids = set()
    rows = []

    if RESUME and os.path.exists(OUT_PATH):
        old = pd.read_csv(OUT_PATH)
        if "event_id" in old.columns:
            done_ids = set(old["event_id"].dropna().astype(int).tolist())
        rows = old.to_dict(orient="records")
        print("resume ON | already in OUT:", len(done_ids))
    else:
        # overwrite mode
        if os.path.exists(OUT_PATH):
            print("resume OFF | will overwrite existing OUT_PATH")

    driver = build_driver(headless=HEADLESS)

    try:
        fetched = 0
        for idx, eid in enumerate(event_ids, 1):
            if RESUME and eid in done_ids:
                continue

            sid = season_map.get(eid) if season_map else None

            stats_json = load_cached(eid)
            if not stats_json:
                url = STATS_URL_TMPL.format(event_id=eid)

                debug = (fetched < 3)  # show first 3 fetches
                stats_json = selenium_fetch_json(driver, url, timeout=20, debug=debug)

                if not stats_json:
                    row = {"event_id": eid, "fetch_ok": 0}
                    if sid is not None:
                        row["season_id"] = sid
                    rows.append(row)
                    print("FAIL", eid, "|", idx, "/", len(event_ids))
                    time.sleep(0.8 + random.random() * 0.6)
                    continue

                save_cached(eid, stats_json)

            flat = flatten_sofascore_statistics(stats_json, period="ALL")
            row = {"event_id": eid, "fetch_ok": 1}
            if sid is not None:
                row["season_id"] = sid
            row.update(flat)
            rows.append(row)

            fetched += 1
            print("OK", eid, "| fetched:", fetched, "| progress:", idx, "/", len(event_ids), "| cols in row:", len(row))

            if fetched % 25 == 0:
                write_rows(rows, OUT_PATH)

            time.sleep(0.8 + random.random() * 0.6)

        write_rows(rows, OUT_PATH)
        print("DONE. saved:", OUT_PATH, "rows:", len(rows))

    finally:
        driver.quit()


if __name__ == "__main__":
    main()


loaded event_ids: 287
resume OFF | will overwrite existing OUT_PATH
OK 14195487 | fetched: 1 | progress: 1 / 287 | cols in row: 89
OK 14195482 | fetched: 2 | progress: 2 / 287 | cols in row: 3
OK 14254081 | fetched: 3 | progress: 3 / 287 | cols in row: 89
OK 14195505 | fetched: 4 | progress: 4 / 287 | cols in row: 89
OK 14254830 | fetched: 5 | progress: 5 / 287 | cols in row: 3
OK 14195502 | fetched: 6 | progress: 6 / 287 | cols in row: 91
OK 14566651 | fetched: 7 | progress: 7 / 287 | cols in row: 113
OK 15200524 | fetched: 8 | progress: 8 / 287 | cols in row: 3
OK 14083164 | fetched: 9 | progress: 9 / 287 | cols in row: 109
OK 14195506 | fetched: 10 | progress: 10 / 287 | cols in row: 3
OK 14195485 | fetched: 11 | progress: 11 / 287 | cols in row: 91
OK 14195518 | fetched: 12 | progress: 12 / 287 | cols in row: 91
OK 15200513 | fetched: 13 | progress: 13 / 287 | cols in row: 3
OK 14195521 | fetched: 14 | progress: 14 / 287 | cols in row: 89
OK 14296929 | fetched: 15 | progress: 15 / 

In [3]:
import os
import re
import csv
import time
import json
import random
import pandas as pd

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import TimeoutException

IN_PATH = "/Users/chenjingxiang/PowerRanking/spl_match_links_3_seasons.csv"
OUT_PATH = "/Users/chenjingxiang/PowerRanking/spl_event_teams_3_seasons.csv"
CACHE_DIR = "/Users/chenjingxiang/PowerRanking/.sofascore_event_page_cache"

HEADLESS = False
RESUME = True          # if OUT_PATH exists, skip event_ids already done
MAX_EVENTS = None      # set small number to test quickly, e.g. 20

def build_driver(headless=False):
    opts = Options()
    if headless:
        opts.add_argument("--headless=new")
    opts.add_argument("--window-size=1400,900")
    opts.add_argument("--disable-gpu")
    opts.add_argument("--no-sandbox")
    opts.add_argument("--disable-dev-shm-usage")
    opts.add_argument("--lang=en-US")
    return webdriver.Chrome(options=opts)

def ensure_dir(path: str):
    os.makedirs(path, exist_ok=True)

def parse_event_id(url: str):
    m = re.search(r"#id:(\d+)", str(url))
    return int(m.group(1)) if m else None

def cache_path(event_id: int):
    return os.path.join(CACHE_DIR, f"{event_id}.json")

def load_cached(event_id: int):
    p = cache_path(event_id)
    if os.path.exists(p):
        try:
            with open(p, "r", encoding="utf-8") as f:
                return json.load(f)
        except Exception:
            return None
    return None

def save_cached(event_id: int, data: dict):
    p = cache_path(event_id)
    try:
        with open(p, "w", encoding="utf-8") as f:
            json.dump(data, f)
    except Exception:
        pass

def scrape_home_away_from_match_page(driver, match_url, timeout=20, debug=False):
    """
    Rule: first team <a> is home, second is away.
    We use a fairly strict selector: anchors that look like team pages.
    If this ever breaks, we can loosen/tune it.
    """
    driver.get(match_url)
    wait = WebDriverWait(driver, timeout)

    # team anchors on SofaScore match page typically link to /football/team/<name>/<id>
    sel = "a[href*='/football/team/']"

    try:
        wait.until(lambda d: len(d.find_elements("css selector", sel)) >= 2)
        links = driver.find_elements("css selector", sel)

        # keep unique by visible text while preserving order
        teams = []
        for a in links:
            t = (a.text or "").strip()
            if t and (len(teams) == 0 or t != teams[-1]):
                teams.append(t)
            if len(teams) >= 2:
                break

        if len(teams) >= 2:
            return teams[0], teams[1], 1  # ok=1
        if debug:
            print("Found <2 team names. URL:", match_url, "teams:", teams)
        return None, None, 0
    except TimeoutException:
        if debug:
            print("Timeout waiting teams. URL:", match_url)
        return None, None, 0

def write_rows(rows, out_path):
    fieldnames = ["season_id", "event_id", "home_team", "away_team", "ok", "match_url"]
    with open(out_path, "w", newline="", encoding="utf-8") as f:
        w = csv.DictWriter(f, fieldnames=fieldnames)
        w.writeheader()
        w.writerows(rows)

def main():
    ensure_dir(CACHE_DIR)

    df = pd.read_csv(IN_PATH)

    # ensure event_id exists
    if "event_id" not in df.columns and "match_url" in df.columns:
        df["event_id"] = df["match_url"].str.extract(r"#id:(\d+)").astype("Int64")

    df = df.dropna(subset=["event_id"]).copy()
    df["event_id"] = df["event_id"].astype(int)

    # minimal columns
    if "season_id" not in df.columns:
        raise ValueError("IN_PATH must contain season_id")

    df = df[["season_id", "event_id", "match_url"]].drop_duplicates()

    if MAX_EVENTS is not None:
        df = df.head(MAX_EVENTS)

    done = set()
    rows = []

    if RESUME and os.path.exists(OUT_PATH):
        old = pd.read_csv(OUT_PATH)
        if "event_id" in old.columns:
            done = set(old["event_id"].dropna().astype(int).tolist())
        rows = old.to_dict(orient="records")
        print("resume ON | already done:", len(done))
    else:
        if os.path.exists(OUT_PATH):
            print("resume OFF | will overwrite:", OUT_PATH)

    driver = build_driver(headless=HEADLESS)

    try:
        total = len(df)
        for i, r in enumerate(df.itertuples(index=False), 1):
            season_id = int(r.season_id)
            event_id = int(r.event_id)
            match_url = str(r.match_url)

            if RESUME and event_id in done:
                continue

            cached = load_cached(event_id)
            if cached:
                home_team = cached.get("home_team")
                away_team = cached.get("away_team")
                ok = cached.get("ok", 0)
            else:
                debug = (i <= 3)
                home_team, away_team, ok = scrape_home_away_from_match_page(
                    driver, match_url, timeout=25, debug=debug
                )
                save_cached(event_id, {"home_team": home_team, "away_team": away_team, "ok": ok})

            out_row = {
                "season_id": season_id,
                "event_id": event_id,
                "home_team": home_team,
                "away_team": away_team,
                "ok": ok,
                "match_url": match_url
            }
            rows.append(out_row)

            if ok == 1:
                print(f"[{i}/{total}] OK  event {event_id} | {home_team} vs {away_team}")
            else:
                print(f"[{i}/{total}] FAIL event {event_id} | url={match_url}")

            # periodic save
            if len(rows) % 25 == 0:
                write_rows(rows, OUT_PATH)

            time.sleep(0.8 + random.random() * 0.6)

        write_rows(rows, OUT_PATH)
        print("DONE. saved:", OUT_PATH, "rows:", len(rows))

    finally:
        driver.quit()

if __name__ == "__main__":
    main()


resume ON | already done: 74
[76/303] OK  event 12246978 | Lion City Sailors vs DPMM
[77/303] OK  event 12247012 | Lion City Sailors vs DPMM
[78/303] OK  event 12247046 | Lion City Sailors vs DPMM
[79/303] OK  event 12404673 | Lion City Sailors vs DPMM
[80/303] OK  event 12246930 | DPMM vs Tampines Rovers
[81/303] OK  event 12246960 | DPMM vs Tampines Rovers
[82/303] OK  event 12246996 | DPMM vs Tampines Rovers
[83/303] OK  event 12247033 | DPMM vs Tampines Rovers
[84/303] OK  event 12246918 | DPMM vs Tanjong Pagar Utd.
[85/303] OK  event 12246965 | DPMM vs Tanjong Pagar Utd.
[86/303] OK  event 12247002 | DPMM vs Tanjong Pagar Utd.
[87/303] OK  event 12247047 | DPMM vs Tanjong Pagar Utd.
[88/303] OK  event 12246944 | Young Lions vs DPMM
[89/303] OK  event 12246989 | Young Lions vs DPMM
[90/303] OK  event 12247017 | Young Lions vs DPMM
[91/303] OK  event 12247063 | Young Lions vs DPMM
[94/303] OK  event 12246945 | Geylang Int. vs Balestier Khalsa
[95/303] OK  event 12246988 | Geylang In

In [9]:
import pandas as pd

MATCH_PATH = "/Users/chenjingxiang/Documents/Y1S1/AnalyticsOnTheCloud/PowerRanking/spl_event_teams_3_seasons.csv"
STATS_PATH = "/Users/chenjingxiang/Documents/Y1S1/AnalyticsOnTheCloud/PowerRanking/spl_stats_3_seasons.csv"
OUT_PATH   = "/Users/chenjingxiang/Documents/Y1S1/AnalyticsOnTheCloud/PowerRanking/spl_full_matches_3_seasons.csv"

# -------------------------
# SPL canonical team list
# -------------------------
SPL_TEAMS = {
    'Albirex Niigata', 'Geylang Int.',
    'Lion City Sailors',
    'Balestier Khalsa', 
    'Tampines Rovers','Hougang Utd',
    'Tanjong Pagar Utd.', 'Young Lions','DPMM'
}

# -------------------------
# Season label map
# -------------------------
SEASON_LABEL = {
    78478: "25/26",
    59708: "2024",
    48772: "2023"
}

# -------------------------
# Load data
# -------------------------
df_match = pd.read_csv(MATCH_PATH)
df_stats = pd.read_csv(STATS_PATH)

print(df_match['home_team'].unique())
# ensure event_id int
df_match["event_id"] = df_match["event_id"].astype(int)
df_stats["event_id"] = df_stats["event_id"].astype(int)

# drop duplicate stats rows
df_stats = df_stats.drop_duplicates(subset=["event_id"])

# -------------------------
# Merge (LEFT JOIN)
# -------------------------
df = df_match.merge(
    df_stats,
    on="event_id",
    how="left",
    suffixes=("", "_stats")
)

# -------------------------
# Add / enforce season_label
# -------------------------
df["season_label"] = df["season_id"].map(SEASON_LABEL)

# -------------------------
# Filter: SPL teams only
# (both home AND away must be SPL)
# -------------------------
df = df[
    df["home_team"].isin(SPL_TEAMS)
].copy()

# -------------------------
# Reorder key columns
# -------------------------
key_cols = [
    "season_id",
    "season_label",
    "event_id",
    "home_team",
    "away_team",
    "fetch_ok",
    "match_url",
]

other_cols = [c for c in df.columns if c not in key_cols]
df = df[key_cols + other_cols]

# -------------------------
# Save
# -------------------------
df.to_csv(OUT_PATH, index=False)

print("saved:", OUT_PATH)
print("rows:", len(df))
print(
    "matches with stats:",
    df["fetch_ok"].fillna(0).astype(int).sum(),
    "/",
    len(df)
)
print(df.head(5))


['Albirex Niigata' 'Geylang Int.' 'Lion City Sailors' 'Balestier Khalsa'
 'Barcelona' 'CF Talavera' 'Alavés' 'Tampines Rovers' 'Guadalajara'
 'Hougang Utd' 'Tanjong Pagar Utd.' 'Inter' 'Real Madrid' 'Young Lions'
 'DPMM']
saved: /Users/chenjingxiang/Documents/Y1S1/AnalyticsOnTheCloud/PowerRanking/spl_full_matches_3_seasons.csv
rows: 279
matches with stats: 279 / 279
   season_id season_label  event_id          home_team         away_team  \
0      78478        25/26  14195487    Albirex Niigata  Balestier Khalsa   
1      78478        25/26  14195482       Geylang Int.   Albirex Niigata   
2      78478        25/26  14254081       Geylang Int.   Albirex Niigata   
3      78478        25/26  14195505  Lion City Sailors   Albirex Niigata   
4      78478        25/26  14254830    Albirex Niigata   Tampines Rovers   

   fetch_ok                                          match_url  ok  \
0         1  https://www.sofascore.com/football/match/albir...   1   
1         1  https://www.sofascore

In [ ]:
#for future fixtures
import re
import csv
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

URL = "https://www.flashscore.com/football/singapore/premier-league/fixtures/"
OUT_PATH = "/Users/chenjingxiang/PowerRanking/spl_flashscore_future_fixtures.csv"


def build_driver(headless=False):
    opts = Options()
    if headless:
        opts.add_argument("--headless=new")
    opts.add_argument("--window-size=1400,900")
    opts.add_argument("--lang=en-US")
    return webdriver.Chrome(options=opts)


def try_accept_cookies(driver):
    # best-effort
    xps = [
        "//button[contains(., 'Accept')]",
        "//button[contains(., 'I agree')]",
        "//button[contains(., 'AGREE')]",
        "//button[contains(., 'OK')]",
    ]
    for xp in xps:
        try:
            b = driver.find_element(By.XPATH, xp)
            if b.is_displayed():
                b.click()
                time.sleep(0.4)
                return
        except Exception:
            pass


def parse_match_id(href: str):
    # Prefer "g_1_<ID>" / "?mid=<ID>"
    if not href:
        return None
    m = re.search(r"[?&]mid=([A-Za-z0-9]+)", href)
    if m:
        return m.group(1)
    m = re.search(r"/match/[^/]+/[^/]+/\\?mid=([A-Za-z0-9]+)", href)
    if m:
        return m.group(1)
    return None


def parse_kickoff_text(txt: str):
    # "30.01. 06:30"
    txt = (txt or "").strip()
    m = re.match(r"(\d{1,2}\.\d{1,2}\.)\s+(\d{1,2}:\d{2})", txt)
    if not m:
        return None, None
    return m.group(1), m.group(2)


def main():
    driver = build_driver(headless=False)
    wait = WebDriverWait(driver, 30)

    try:
        driver.get(URL)
        try_accept_cookies(driver)

        # Wait until at least one row shows up
        wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "a.eventRowLink")))

        # Scroll to load more (fixtures can be lazy-loaded)
        last = 0
        stable = 0
        for _ in range(40):
            links = driver.find_elements(By.CSS_SELECTOR, "a.eventRowLink")
            if len(links) == last:
                stable += 1
            else:
                stable = 0
                last = len(links)

            if stable >= 3:
                break

            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(0.8)

        # Each match row contains: a.eventRowLink + homeParticipant + awayParticipant + event__time
        # We'll iterate over anchors and resolve siblings via JS to stay close to your snippet.
        anchors = driver.find_elements(By.CSS_SELECTOR, "a.eventRowLink")
        rows = []
        seen = set()

        for a in anchors:
            href = a.get_attribute("href") or ""
            mid = parse_match_id(href)

            # Extract home/away/time from DOM near this anchor
            data = driver.execute_script("""
                const a = arguments[0];

                function findNext(el, selector){
                    let cur = el;
                    for (let i=0; i<12; i++){
                        if (!cur) return null;
                        cur = cur.nextElementSibling;
                        if (!cur) return null;
                        if (cur.matches(selector)) return cur;
                    }
                    return null;
                }

                const homeDiv = findNext(a, "div.event__homeParticipant");
                const awayDiv = findNext(a, "div.event__awayParticipant");
                const timeDiv = findNext(a, "div.event__time");

                const home = homeDiv ? (homeDiv.querySelector("span[data-testid='wcl-scores-simple-text-01']")?.innerText || "").trim() : "";
                const away = awayDiv ? (awayDiv.querySelector("span[data-testid='wcl-scores-simple-text-01']")?.innerText || "").trim() : "";
                const t = timeDiv ? (timeDiv.innerText || "").trim() : "";

                return {home, away, t};
            """, a)

            home = (data.get("home") or "").strip()
            away = (data.get("away") or "").strip()
            time_txt = (data.get("t") or "").strip()
            date_ddmm, kickoff_time = parse_kickoff_text(time_txt)

            # de-dupe key
            key = (mid, href, home, away, time_txt)
            if key in seen:
                continue
            seen.add(key)

            rows.append({
                "flashscore_match_id": mid,
                "match_url": href,
                "home_team": home,
                "away_team": away,
                "date_ddmm": date_ddmm,     # e.g. "30.01."
                "kickoff_time": kickoff_time,  # e.g. "06:30"
                "time_raw": time_txt
            })

        # Save
        fieldnames = ["flashscore_match_id", "match_url", "home_team", "away_team",
                      "date_ddmm", "kickoff_time", "time_raw"]

        with open(OUT_PATH, "w", newline="", encoding="utf-8") as f:
            w = csv.DictWriter(f, fieldnames=fieldnames)
            w.writeheader()
            w.writerows(rows)

        print("saved:", OUT_PATH, "rows:", len(rows))
        print("sample:", rows[:3])

    finally:
        driver.quit()


if __name__ == "__main__":
    main()


saved: /Users/chenjingxiang/PowerRanking/spl_flashscore_future_fixtures.csv rows: 65
sample: [{'flashscore_match_id': 'zwNinCpI', 'match_url': 'https://www.flashscore.com/match/football/balestier-khalsa-lK4UsSns/geylang-SGUUj41C/?mid=zwNinCpI', 'home_team': 'Geylang', 'away_team': 'Balestier Khalsa', 'date_ddmm': '09.01.', 'kickoff_time': '06:30', 'time_raw': '09.01. 06:30'}, {'flashscore_match_id': 'r3QapjFU', 'match_url': 'https://www.flashscore.com/match/football/albirex-niigata-2TtIKp7i/young-lions-vsNto8vQ/?mid=r3QapjFU', 'home_team': 'Albirex Niigata', 'away_team': 'Young Lions', 'date_ddmm': '10.01.', 'kickoff_time': '06:30', 'time_raw': '10.01. 06:30'}, {'flashscore_match_id': 'MTeA4g0n', 'match_url': 'https://www.flashscore.com/match/football/lion-city-sailors-OnKeiOGI/tampines-KbVkFR8P/?mid=MTeA4g0n', 'home_team': 'Tampines', 'away_team': 'Lion City', 'date_ddmm': '11.01.', 'kickoff_time': '06:30', 'time_raw': '11.01. 06:30'}]


In [11]:
#gather event id for future fixtures for polling
# get links by clicking the RIGHT (next round) button
import re
import time
import csv
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import JavascriptException

START_URL = "https://www.sofascore.com/tournament/football/singapore/premier-league/634"
SEASON_IDS = [78478]

OUT_PATH = "spl_match_links_3_seasons_next.csv"

def build_driver(headless=False):
    opts = Options()
    if headless:
        opts.add_argument("--headless=new")
    opts.add_argument("--window-size=1400,900")
    opts.add_argument("--disable-gpu")
    opts.add_argument("--no-sandbox")
    opts.add_argument("--disable-dev-shm-usage")
    opts.add_argument("--lang=en-US")
    return webdriver.Chrome(options=opts)

def parse_event_id(url: str):
    m = re.search(r"#id:(\d+)", url)
    return int(m.group(1)) if m else None

def js_collect_links(driver):
    return driver.execute_script("""
        const out = [];
        const anchors = document.querySelectorAll("a[href*='/football/match/']");
        for (const a of anchors) {
            const href = a.getAttribute("href");
            if (!href) continue;
            if (href.includes("#id:")) out.push(new URL(href, location.origin).href);
        }
        return Array.from(new Set(out));
    """)

def wait_match_list_present(driver, timeout=20):
    wait = WebDriverWait(driver, timeout)
    wait.until(lambda d: d.execute_script(
        "return document.querySelectorAll(\"a[href*='/football/match/']\").length"
    ) > 0)

def js_click_next_round(driver):
    """
    Click the RIGHT arrow button that loads newer rounds.
    """
    try:
        return driver.execute_script(r"""
            const sig = "M18 12.01 9.942 20 8.51 18.58l6.636-6.57L8.5 5.41 9.922 4z";
            const paths = Array.from(document.querySelectorAll("button svg path"));

            for (const p of paths) {
                const d = (p.getAttribute("d") || "").trim();
                if (d === sig) {
                    const btn = p.closest("button");
                    if (!btn) continue;

                    const ariaDisabled = btn.getAttribute("aria-disabled");
                    const isDisabled = btn.disabled || ariaDisabled === "true" || btn.hasAttribute("disabled");
                    if (isDisabled) return false;

                    btn.scrollIntoView({block: "center"});
                    btn.click();
                    return true;
                }
            }
            return false;
        """)
    except JavascriptException:
        return False

def scrape_one_season_by_clicking_next(driver, season_id, max_clicks=80, pause=0.9, stop_when_no_new=5):
    driver.get(f"{START_URL}#id:{season_id}")
    wait_match_list_present(driver)

    all_links = set()
    no_new = 0

    # initial collect
    for u in js_collect_links(driver):
        all_links.add(u)

    for i in range(max_clicks):
        before = len(all_links)

        clicked = js_click_next_round(driver)
        if not clicked:
            print(f"season {season_id} | NEXT button disabled/not found -> stop")
            break

        time.sleep(pause)

        for u in js_collect_links(driver):
            all_links.add(u)

        after = len(all_links)
        print(f"season {season_id} | click {i+1} | links {after}")

        if after == before:
            no_new += 1
        else:
            no_new = 0

        if no_new >= stop_when_no_new:
            print(f"season {season_id} | no new links for {stop_when_no_new} clicks -> stop")
            break

    rows = []
    for u in sorted(all_links):
        eid = parse_event_id(u)
        if eid is not None:
            rows.append({"season_id": season_id, "event_id": eid, "match_url": u})

    # de-dupe by (season_id, event_id)
    seen = set()
    uniq = []
    for r in rows:
        key = (r["season_id"], r["event_id"])
        if key not in seen:
            seen.add(key)
            uniq.append(r)

    return uniq

def save_csv(rows, out_path=OUT_PATH):
    with open(out_path, "w", newline="", encoding="utf-8") as f:
        w = csv.DictWriter(f, fieldnames=["season_id", "event_id", "match_url"])
        w.writeheader()
        w.writerows(rows)
    print("saved:", out_path, "rows:", len(rows))

if __name__ == "__main__":
    driver = build_driver(headless=False)
    try:
        all_rows = []
        for sid in SEASON_IDS:
            all_rows.extend(scrape_one_season_by_clicking_next(driver, sid))

        # global de-dupe
        seen = set()
        final_rows = []
        for r in all_rows:
            key = (r["season_id"], r["event_id"])
            if key not in seen:
                seen.add(key)
                final_rows.append(r)

        print("TOTAL:", len(final_rows))
        print("SAMPLE:", final_rows[:5])
        save_csv(final_rows)

    finally:
        driver.quit()


KeyboardInterrupt: 